In [2]:
import gym
import highway_env
import numpy as np
import tensorflow as tf

## Create a environment

In [93]:

env = gym.make("highway-v0")

c:\Users\gabri\Desktop\ml\highway_driver\env\lib\site-packages\gym\utils\passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(


In [36]:
env.observation_space.sample()

array([[-1.19634286e-01,  1.32399714e+00, -1.08036447e+00,
         1.06072640e+00,  4.57237095e-01],
       [ 3.21855754e-01,  8.53351712e-01, -2.77275443e-01,
         1.96413766e-03, -7.04319417e-01],
       [-7.46680021e-01,  4.32915270e-01, -3.25910509e-01,
        -4.10995096e-01,  7.90312290e-01],
       [ 2.36081943e-01,  8.44408631e-01,  6.72127604e-01,
        -1.73742807e+00,  1.98495674e+00],
       [-1.54920232e+00, -1.44350970e+00, -7.44584560e-01,
         4.41996634e-01, -1.59329787e-01]], dtype=float32)

In [37]:
env.action_space.sample()

3

In [62]:
env.config["simulation_frequency"] = 3

In [94]:
episodes = 2
for ep in range(1, episodes+1):
    done = truncated = False
    score = 0
    obs = env.reset()
    while not (done or truncated):
        action = env.action_space.sample() # Your agent code here
        obs, reward, done, info = env.step(action)
        score += reward
        env.render()
    print('Episode:{} Score:{}'.format(ep, score))
env.close()

c:\Users\gabri\Desktop\ml\highway_driver\env\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\gabri\Desktop\ml\highway_driver\env\lib\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
c:\Users\gabri\Desktop\ml\highway_driver\env\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
c:\Users\gabri\Desktop\ml\highway_driver\env\lib\site-packages\gym\utils\passi

Episode:1 Score:7.580389651603979
Episode:2 Score:4.305648556771382


In [64]:
env.close()

## Create PPO Model

In [65]:
from stable_baselines3 import PPO
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy
import os

In [66]:
path_tensorboard = os.path.join("Training", "Logs")

In [29]:
path_tensorboard

'Training\\Logs'

In [69]:
model = PPO("MlpPolicy", env=env,
            batch_size=64,
            n_steps=128,
            n_epochs=10,
            verbose=1, 
            tensorboard_log=path_tensorboard)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [20]:
model_dqn = DQN('MlpPolicy', env,
              policy_kwargs=dict(net_arch=[256, 256]),
              learning_rate=5e-4,
              buffer_size=15000,
              learning_starts=200,
              batch_size=32,
              gamma=0.8,
              train_freq=1,
              gradient_steps=1,
              target_update_interval=50,
              verbose=1,
              tensorboard_log=path_tensorboard)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [22]:
model_dqn.learn(total_timesteps=10000)

Logging to highway_dqn/DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10.2     |
|    ep_rew_mean      | 8.57     |
|    exploration_rate | 0.961    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1        |
|    time_elapsed     | 28       |
|    total_timesteps  | 41       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10.5     |
|    ep_rew_mean      | 8.25     |
|    exploration_rate | 0.92     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1        |
|    time_elapsed     | 69       |
|    total_timesteps  | 84       |
----------------------------------


KeyboardInterrupt: 

In [70]:
model.learn(total_timesteps=10000)

Logging to Training\Logs\PPO_8
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 10.2     |
|    ep_rew_mean     | 7.51     |
| time/              |          |
|    fps             | 22       |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 128      |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 11.3         |
|    ep_rew_mean          | 8.42         |
| time/                   |              |
|    fps                  | 21           |
|    iterations           | 2            |
|    time_elapsed         | 11           |
|    total_timesteps      | 256          |
| train/                  |              |
|    approx_kl            | 0.0029070205 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.61        |
|    explained_variance   |

## Evaluate the model

In [99]:
env.config["simulation_frequency"] = 15

In [100]:
int(model.predict(obs)[0])

4

In [101]:
episodes = 2
for ep in range(1, episodes+1):
    done = truncated = False
    score = 0
    obs = env.reset()
    while not (done or truncated):
        action = int(model.predict(obs)[0])
        obs, reward, done, info = env.step(action)
        score += reward
        env.render()
    print('Episode:{} Score:{}'.format(ep, score))
env.close()

Episode:1 Score:19.577876679067852
Episode:2 Score:28.457276524644957


### Change the difficult

In [103]:
env.config["high_speed_reward"] = 3 ## Reward for driving at high speed

In [105]:
model_fast = PPO("MlpPolicy", env=env, verbose=1, n_steps=128, tensorboard_log=path_tensorboard)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [107]:
model_fast.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_10
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 9.31     |
|    ep_rew_mean     | 4.69     |
| time/              |          |
|    fps             | 6        |
|    iterations      | 1        |
|    time_elapsed    | 20       |
|    total_timesteps | 128      |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 10.5         |
|    ep_rew_mean          | 5.86         |
| time/                   |              |
|    fps                  | 6            |
|    iterations           | 2            |
|    time_elapsed         | 37           |
|    total_timesteps      | 256          |
| train/                  |              |
|    approx_kl            | 0.0034607872 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.61        |
|    explained_variance   

In [ ]:
path_save_fast = os.path.join("Training", "model_fast")

In [ ]:
model_fast.save("highway_fast")